In [26]:
!pip install langchain_core
!pip install langchain
!pip install langchain_community
!pip install arxiv
!pip install pypdf
!pip install faiss-gpu
!pip install groq
!pip install fastembed
!pip install wikipedia
!pip install python-dotenv
!pip install langchain-groq
!pip install sentence-transformers
!pip install -U langchain-huggingface

In [27]:
!pip install pypdf

In [28]:
!pip install langchainhub==0.1.15

In [29]:
import os
from dotenv import load_dotenv
load_dotenv()

False

In [30]:
from langchain_groq import ChatGroq
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun

In [31]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from fastembed import TextEmbedding
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.document_loaders import DirectoryLoader

In [32]:
import os
from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
api = os.getenv("GROQ_API_KEY")
Model = "llama-3.1-70b-versatile"
llm = ChatGroq(api_key=api,model=Model)

In [33]:
def load_pdf(data):
  loader = DirectoryLoader(data,glob="*.pdf",loader_cls = PyPDFLoader)
  documents = loader.load()
  return documents

extracted_data = load_pdf("data/")

In [34]:
def text_split(extracted_data):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlap=200)
  text_chunks = text_splitter.split_documents(extracted_data)
  return text_chunks

In [35]:
text_chunks = text_split(extracted_data)
print(f"Length of the Chunk {len(text_chunks)}")

Length of the Chunk 4445


In [36]:
def embedding_model():
  model_name = "sentence-transformers/all-MiniLM-L6-v2"
  embedding = HuggingFaceEmbeddings(model_name = model_name)
  return embedding

In [37]:
embeddings = embedding_model()

In [38]:
sample_embedding = embeddings.embed_query("CDK")
print(f"Length of the sample_embedding{len(sample_embedding)}")
print(f"Example of how the embedding model looks like : {sample_embedding}")

Length of the sample_embedding384
Example of how the embedding model looks like : [-0.07624874264001846, 0.11039117723703384, -0.08735432475805283, -0.037289027124643326, -0.03572263568639755, -0.059426192194223404, 0.09830831736326218, 0.07049110531806946, 0.014280569739639759, 0.03800587356090546, 0.0020426714327186346, -0.10246293991804123, 0.015605214051902294, -0.031947873532772064, -0.012786182574927807, 0.042728241533041, -0.01973150297999382, 0.008957058191299438, -0.06602046638727188, -0.031694408506155014, -0.11810354143381119, -0.04171375185251236, -0.024348357692360878, -0.015065202489495277, -0.02605675905942917, -0.016803612932562828, 0.03977925330400467, 0.07697451859712601, -0.05818972736597061, -0.0630159005522728, -0.000793376297224313, 0.03802512586116791, 0.02524895779788494, 0.002769293962046504, 0.026908647269010544, 0.056114643812179565, -0.04573231190443039, -0.014879941940307617, -0.02331913262605667, -0.039954543113708496, 0.01325595285743475, -0.0584378279745

In [39]:
def vector_embedding():
  store = FAISS.from_documents(text_chunks,embeddings)
  embedding = store.as_retriever()
  return embedding
vector_database = vector_embedding()

In [40]:
prompt_template = """
Hello mister You are the AI, your name is BrainWave okayy then if user asks the question related to health , medicine or like related field you need to give the related answer only clearly check the answer whether it is correct or not
if it's not correct just inform BrainWave AI doesn't know about this and don't make up an answer and say I don't know about this in a beautiful manner
context:{context}
question:{question}
"""

In [41]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate(template = prompt_template , input_variables=["context","query"],chain_type_kwargs={"prompt":prompt_template})

In [42]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(llm = llm,
            chain_type="stuff",
            retriever=vector_database,
            chain_type_kwargs={"prompt":prompt})

In [43]:
user_input = input("Input Prompt: ")
result = qa_chain({"query": user_input})
final_result = result["result"]
print(final_result)

Input Prompt: Helloooo
Hello, it's nice to meet you. I'm BrainWave, your AI assistant. I understand that I'll be helping you with questions related to health, medicine, and related fields. I'll do my best to provide accurate and clear answers. If I'm unsure or don't know the answer to a question, I'll let you know in a nice way, saying "BrainWave AI doesn't know about this" or something similar. Please feel free to ask me any questions you have.


#GOOGLE SEARCH API

In [44]:
!pip install langchainhub
!pip install huggingface_hub

In [51]:
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_core.tools import Tool

os.environ["GOOGLE_CSE_ID"] = userdata.get("GOOGLE_CSE_ID")
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

In [52]:
google_search = GoogleSearchAPIWrapper()

google_tool = Tool(
    name = "google-search",
    description = "Search for medical and health related queries only",
    func = google_search.run
)

In [60]:
google_search.run("How much time it take to get releif from lumbar l5 Spondylolysis")

'How is spondylolysis treated? Your provider will treat your pain and other symptoms to give the pars fracture time to heal. · How long does spondylolysis take to\xa0... Sep 7, 2022 ... When do I need surgery for it? What is the recovery time? What is spondylolysis? Spondylolysis is a spinal defect or fracture of a bone\xa0... How long do lumbar epidural injections last for back pain? Studies on lumbar ESIs have revealed that they can provide reliable pain relief for up to 6 months in\xa0... Apr 17, 2023 ... ... time. Dorsal and lateral ... Many cases of spondylosis are effectively treated with physical therapy and pain relief measures. It will take about 4 to 6 weeks for you to reach your expected level of ... How long you need to stay in hospital will depend on the specific type of\xa0... Aug 7, 2023 ... [3][11][25] Additionally, in the setting of retrolisthesis and disc herniation at L5-S1, recent long-term follow-up from the SPORT database has\xa0... ... take several weeks, but if 

In [66]:
from langchain import hub
prompt = hub.pull("hwchase17/structured-chat-agent")

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:333: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [68]:
from langchain.agents import create_structured_chat_agent
agent = create_structured_chat_agent(
    llm=llm,
    tools=[google_tool],
    prompt=prompt,
)

In [ ]:
from langchain.agents import AgentE